# 实验参数设置

In [1]:
from FederatedLearning.learner import CNNModel

clients_num = 10 # 客户端总数
rotation_cycle = 5 # 角色轮换间隔轮数
max_round = 20 # 联邦学习轮数
learning_rate = 0.01
model_class = CNNModel # 选用CNN模型

## 数据切分

In [2]:
import data_split
# 使用data_split模块中的函数获取PyTorch数据集
train_dataset = data_split.get_mnist_pytorch_dataset(train=True)
test_dataset = data_split.get_mnist_pytorch_dataset(train=False)
# test
# 使用正确的数据集对象创建数据加载器
client_dataloaders = data_split.create_client_dataloaders(train_dataset, clients_num // 2, 64, num_workers=clients_num) * 2
client_test_loader = data_split.create_client_dataloaders(test_dataset, clients_num // 2, 64) * 2

2025-10-22 19:37:57,227 - DGS_BCFL - INFO - 
切分5份数据集...
2025-10-22 19:37:57,229 - DGS_BCFL - INFO -  1 数据准备完成，样本数: 12000
2025-10-22 19:37:57,230 - DGS_BCFL - INFO -  2 数据准备完成，样本数: 12000
2025-10-22 19:37:57,232 - DGS_BCFL - INFO -  3 数据准备完成，样本数: 12000
2025-10-22 19:37:57,233 - DGS_BCFL - INFO -  4 数据准备完成，样本数: 12000
2025-10-22 19:37:57,234 - DGS_BCFL - INFO -  5 数据准备完成，样本数: 12000
2025-10-22 19:37:57,235 - DGS_BCFL - INFO - 
切分5份数据集...
2025-10-22 19:37:57,235 - DGS_BCFL - INFO -  1 数据准备完成，样本数: 2000
2025-10-22 19:37:57,236 - DGS_BCFL - INFO -  2 数据准备完成，样本数: 2000
2025-10-22 19:37:57,236 - DGS_BCFL - INFO -  3 数据准备完成，样本数: 2000
2025-10-22 19:37:57,237 - DGS_BCFL - INFO -  4 数据准备完成，样本数: 2000
2025-10-22 19:37:57,237 - DGS_BCFL - INFO -  5 数据准备完成，样本数: 2000


## 开始训练

In [ ]:
from owner import Owner
from client import Client
    
    
# 初始化管理者，并获得初始化字典
owner = Owner(rotation_cycle=1, model_class=model_class)
main_dict = owner.get_main_dict()

# 初始化客户端以及身份
clients = [owner]
for i in range(clients_num):
    client_name = f"client_{i + 1}"
    client = Client(epochs=2, client_name=client_name, data_loader=client_dataloaders[i],
                    test_loader=client_test_loader[i], ModelClass=model_class, main_dict=main_dict, learning_rate=learning_rate)
    owner.join(client_name)
    clients.append(client)

import threading

for _ in range(max_round):
    t = [threading.Thread(target=client.run) for client in clients]
    _ = [i.start() for i in t]
    _ = [i.join() for i in t]

    


# 保存实验过程和结果
import pickle
with open("main_dict.pkl", "wb") as f:
    pickle.dump(main_dict, f)

2025-10-22 19:44:17,060 - DGS_BCFL - INFO - [client_2] 当前轮次角色: aggregator
2025-10-22 19:44:17,061 - DGS_BCFL - INFO - [client_3] 当前轮次角色: validator
2025-10-22 19:44:17,064 - DGS_BCFL - INFO - [client_4] 当前轮次角色: validator
2025-10-22 19:44:17,066 - DGS_BCFL - INFO - [client_5] 当前轮次角色: validator
2025-10-22 19:44:17,069 - DGS_BCFL - INFO - [client_6] 当前轮次角色: learner
2025-10-22 19:44:17,070 - DGS_BCFL - INFO - [client_7] 当前轮次角色: learner
2025-10-22 19:44:17,073 - DGS_BCFL - INFO - [client_8] 当前轮次角色: learner
2025-10-22 19:44:17,077 - DGS_BCFL - INFO - FederatedLearner使用设备: cuda
2025-10-22 19:44:17,077 - DGS_BCFL - INFO - [client_9] 当前轮次角色: learner
2025-10-22 19:44:17,078 - DGS_BCFL - INFO - [client_10] 当前轮次角色: learner
2025-10-22 19:44:17,089 - DGS_BCFL - INFO - [client_3] 验证者初始化...
2025-10-22 19:44:17,091 - DGS_BCFL - INFO - Aggregator使用设备: cuda
2025-10-22 19:44:17,098 - DGS_BCFL - INFO - [client_4] 验证者初始化...
2025-10-22 19:44:17,105 - DGS_BCFL - INFO - FederatedLearner使用设备: cuda
2025-10-22 19:

## 计算结果可视化

In [4]:
import pickle
from matplotlib import pyplot as plt
import numpy as np
# 读取实验结果
main_dict = pickle.load(open("main_dict.pkl", "rb"))

# 精确度变化可视化
global_accuracy_history = main_dict["global_accuracy_history"]
# 创建图形
plt.figure(figsize=(12, 7))

# 绘制主要曲线
epochs = list(range(1, len(global_accuracy_history) + 1))
plt.plot(epochs, global_accuracy_history, 'b-', linewidth=2.5, marker='o', markersize=6, 
         markerfacecolor='red', markeredgecolor='darkred', markeredgewidth=1, 
         label='模型准确度')
# 显示图表
plt.show()
    


draw_data(global_accuracy_history)

